In [1]:
from scipy.io.wavfile import read as read_wav
from IPython.display import Audio
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as skl
import librosa as lr
import numpy as np
import logging
import random
import scipy
import os

In [2]:
np.random.seed(5)
logging.warnings.filterwarnings('ignore')

In [3]:
dataset_root = '..'

### Genre Dictionary:

In [4]:
genres = {
    'strong': 0,
    'mild': 1
}

### Hyper Parameters:

In [5]:
hparams = {
    'samplerate': 22050,
    'seq_length': 256,
    'hop_length': 512,
    'fft_window': 2048,
    'num_classes': len(genres)
}

### Extract Features:

In [6]:
def extract_features(filepath, hparams):
    
    audio_data_array = lr.load(filepath, sr=hparams['samplerate'], mono=True)
    
    # normalized = lr.util.normalize(audio_data_array[0])
    
    features = [
        
        # MFCC Features:
        lr.feature.mfcc(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),

        # Chroma STFT:
        lr.feature.chroma_stft(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Centroid:
        lr.feature.spectral_centroid(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Contrast:
        lr.feature.spectral_contrast(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        )
    ]
    
    return np.vstack(features)

### Strong Classes Array:

In [7]:
strong_classes = [
    'hiphop',
    'metal',
    'rock',
    'pop',
    'reggae'
]

### Mild Classes Array:

In [8]:
mild_classes = [
    'disco',
    'country',
    'jazz',
    'classical',
    'blues'
]

### Allocating Arrays to Store Dataset:

In [9]:
train_data_x = np.zeros(shape=(5 * 700, 40, hparams['seq_length']))
val_data_x = np.zeros(shape=(5 * 200, 40, hparams['seq_length']))
test_data_x = np.zeros(shape=(5 * 100, 40, hparams['seq_length']))

train_data_y = np.zeros(shape=(5 * 700, 2))
val_data_y = np.zeros(shape=(5 * 200, 2))
test_data_y = np.zeros(shape=(5 * 100, 2))

### Train Dataset:

In [10]:
preprocess = True

if preprocess == True:

    train_data_counter = 0
    val_data_counter = 0
    test_data_counter = 0

    for genre in strong_classes:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)

        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                test_data_counter += 1
        

    for genre in mild_classes:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)

        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                test_data_counter += 1
            
    if 'prepared' not in os.listdir('.'):
        os.mkdir('prepared')
    np.save('./prepared/train_data_x.npy', train_data_x, allow_pickle=False)
    np.save('./prepared/train_data_y.npy', train_data_y, allow_pickle=False)
    np.save('./prepared/val_data_x.npy', val_data_x, allow_pickle=False)
    np.save('./prepared/val_data_y.npy', val_data_y, allow_pickle=False)
    np.save('./prepared/test_data_x.npy', test_data_x, allow_pickle=False)
    np.save('./prepared/test_data_y.npy', test_data_y, allow_pickle=False)
else:
    train_data_x = np.load('./prepared/train_data_x.npy', allow_pickle=False)
    train_data_y = np.load('./prepared/train_data_y.npy', allow_pickle=False)
    val_data_x = np.load('./prepared/val_data_x.npy', allow_pickle=False)
    val_data_y = np.load('./prepared/val_data_y.npy', allow_pickle=False)
    test_data_x = np.load('./prepared/test_data_x.npy', allow_pickle=False)
    test_data_y = np.load('./prepared/test_data_y.npy', allow_pickle=False)

Genre: hiphop Processed.
Genre: metal Processed.
Genre: rock Processed.
Genre: pop Processed.
Genre: reggae Processed.
Genre: disco Processed.
Genre: country Processed.
Genre: jazz Processed.
Genre: classical Processed.
Genre: blues Processed.


In [11]:
train_data_x  = np.moveaxis(train_data_x, 1, 2)
val_data_x    = np.moveaxis(val_data_x,   1, 2)
test_data_x   = np.moveaxis(test_data_x,  1, 2)

### Model Creation and Training:

In [12]:
class GenreClassifierModel(tf.keras.Model):
    
    def __init__(self, l1, l2):
        
        super().__init__()
        
        self.L1 = tf.keras.layers.LSTM(l1, return_sequences=True)
        self.L2 = tf.keras.layers.LSTM(l2, return_sequences=False)
        self.L3 = tf.keras.layers.Dense(units=hparams['num_classes'], activation="softmax")
        
    def call(self, inputs):
        self.x = self.L1(inputs)
        self.x = self.L2(self.x)
        self.x = self.L3(self.x)
        return self.x

### Training Model:

In [13]:
def create_model(config):
    
    l1 = config['L1']
    l2 = config['L2']
    
    # Model Definition:
    model = GenreClassifierModel(l1, l2)
    
    # Model Compilation:
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
model = create_model({
    'L1': 64,
    'L2': 32
})

In [15]:
hist = model.fit(train_data_x, train_data_y, epochs=200)

Epoch 1/200
110/110 [==============================] - 2s 15ms/step - loss: 0.5714 - accuracy: 0.6963
Epoch 2/200
110/110 [==============================] - 2s 16ms/step - loss: 0.5297 - accuracy: 0.7214
Epoch 3/200
110/110 [==============================] - 2s 16ms/step - loss: 0.5088 - accuracy: 0.7340
Epoch 4/200
110/110 [==============================] - 2s 16ms/step - loss: 0.4912 - accuracy: 0.7469
Epoch 5/200
110/110 [==============================] - 2s 16ms/step - loss: 0.4832 - accuracy: 0.7497
Epoch 6/200
110/110 [==============================] - 2s 16ms/step - loss: 0.4747 - accuracy: 0.7566
Epoch 7/200
110/110 [==============================] - 2s 16ms/step - loss: 0.4622 - accuracy: 0.7740
Epoch 8/200
110/110 [==============================] - 2s 16ms/step - loss: 0.4526 - accuracy: 0.7723
Epoch 9/200
110/110 [==============================] - 2s 16ms/step - loss: 0.4433 - accuracy: 0.7820
Epoch 10/200
110/110 [==============================] - 2s 16ms/step - loss: 0.445

110/110 [==============================] - 2s 16ms/step - loss: 0.2484 - accuracy: 0.8883
Epoch 81/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2436 - accuracy: 0.8966
Epoch 82/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2635 - accuracy: 0.8857
Epoch 83/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2346 - accuracy: 0.8997
Epoch 84/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2188 - accuracy: 0.9011
Epoch 85/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2349 - accuracy: 0.8946
Epoch 86/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2404 - accuracy: 0.9014
Epoch 87/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2360 - accuracy: 0.8951
Epoch 88/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2295 - accuracy: 0.9057
Epoch 89/200
110/110 [==============================] - 2s 16ms/step - loss: 0.2220 - 

110/110 [==============================] - 2s 17ms/step - loss: 0.1235 - accuracy: 0.9469
Epoch 160/200
110/110 [==============================] - 2s 18ms/step - loss: 0.1092 - accuracy: 0.9583
Epoch 161/200
110/110 [==============================] - 2s 19ms/step - loss: 0.1044 - accuracy: 0.9609
Epoch 162/200
110/110 [==============================] - 2s 18ms/step - loss: 0.1199 - accuracy: 0.9529
Epoch 163/200
110/110 [==============================] - 2s 20ms/step - loss: 0.1117 - accuracy: 0.9560
Epoch 164/200
110/110 [==============================] - 2s 19ms/step - loss: 0.1148 - accuracy: 0.9583
Epoch 165/200
110/110 [==============================] - 2s 18ms/step - loss: 0.1115 - accuracy: 0.9569
Epoch 166/200
110/110 [==============================] - 2s 19ms/step - loss: 0.1018 - accuracy: 0.9609
Epoch 167/200
110/110 [==============================] - 2s 18ms/step - loss: 0.0993 - accuracy: 0.9623
Epoch 168/200
110/110 [==============================] - 2s 17ms/step - loss: 

In [16]:
model.evaluate(val_data_x, val_data_y)

32/32 [==============================] - 0s 9ms/step - loss: 0.6358 - accuracy: 0.8420


[0.6358094215393066, 0.8420000076293945]

In [17]:
model.save('./models/model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/model/assets
